# Chapter 2 Gaussian Elimination in Detail (Optional)

We investigate the $LU$-decomposition by the Gaussian elimination. For the choice of pivot entries, we adopt the so-called partial pivoting, which choose an entry with a largest absolute value among candidates to enhance numerical stability by dividing by a number as large as possible. We implement the Gaussian elimination in detail, and the audience can see how the matrix changes step by step as the Gaussian elimination progresses. We also confirm that ours produces the same output for $LU$-decomposition as the numpy.linalg.lu function for an example of size $10 \times 10$.

In [ ]:
# numerical and scientific computing libraries 
import numpy as np
import scipy as sp

# plotting libraries
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

In [10]:
# for pretty printing
np.set_printoptions(4, linewidth=100, suppress=True)

--------------------------------------------------------------------------------------------------------------

Let us have a more in-depth look on Gaussian elimination. We already know that performing Gaussian elimination on $A$ is nothing but multiplying an appropriate permutation matrice to the left and lower triangular elementary matrices. However, sometimes, we don't know which rows should be swapped in advance. Furthermore, in a so-called partial pivoting method to enhace numercial stablility, we choose an entry with biggest absolute value among pivot cadidates for each column. Thefore, for $j$-th column, we decide two rows to swap and a correponding permutation matrix $Q_j$ with a lower triangular matrix $L_j$ representing the elimination of entries below the pivot. So the Gaussian elimination under partial pivoting can be expressed by  
\begin{align*} L_k Q_{k-1} L_{k-1} \cdots Q_1 L_1 Q_0 A = U \end{align*}
for some upper triangular $U$. Notice that $Q_j$ can be an identity matrix, if we do not perform a row exchange in the $j$-th step. One observation here is the rows swapped in $Q_j$ are below $j$-th row and $L_j$ have non-zero off-diagonal entries lie in up to the $j$-th column. Therefore, the non-zero pattern of $L_j' = (Q_{k-1} \cdots Q_j) L_j (Q_{k-1} \cdots Q_j)^\top$ coincides with that of $L_j$, which means we can re-write the decomposition as
\begin{align*} L_k L_{k-1}' \cdots L_1' Q_{k-1} \cdots Q_1 Q_0 A = LQA = U. \end{align*}
The triplet $(Q^\top, L^{-1}, U)$ is a usual output of numpy.linalg.lu function. Let us demonstrate this procedure for a randomly generated matrix.

--------------------------------------------------------------------------------------------------------------

In [11]:
n = m = 10
rng = np.random.RandomState(0)

A = rng.randint(10, size=(n, m))
A = A.astype(np.float64)
print(A)

[[5. 0. 3. 3. 7. 9. 3. 5. 2. 4.]
 [7. 6. 8. 8. 1. 6. 7. 7. 8. 1.]
 [5. 9. 8. 9. 4. 3. 0. 3. 5. 0.]
 [2. 3. 8. 1. 3. 3. 3. 7. 0. 1.]
 [9. 9. 0. 4. 7. 3. 2. 7. 2. 0.]
 [0. 4. 5. 5. 6. 8. 4. 1. 4. 9.]
 [8. 1. 1. 7. 9. 9. 3. 6. 7. 2.]
 [0. 3. 5. 9. 4. 4. 6. 4. 4. 3.]
 [4. 4. 8. 4. 3. 7. 5. 5. 0. 1.]
 [5. 9. 3. 0. 5. 0. 1. 2. 4. 2.]]


In [12]:
def elimination(A, eps, verbose):
    A = A.copy()
    n, m = A.shape
    Q_list = []
    L_list = []

    row_to_check = 0

    for j in range(m):
        # We test column by column to see if we can find a pivot.
        if row_to_check >= n :
            # Elimination is done on all n rows, even though there are
            # some columns not yet investigated. Anyway, task completed.
            break

        pivot = row_to_check
        for i in range(row_to_check+1, n):
            # We are exchanging rows "aggressively", so that the pivot
            # is always the largest possible among viable options.
            # This is because we will soon perform division by the pivot,
            # and dividing by a small number is susceptible to errors:
            # compare how different 1/0.0101 and 1/0.01 are,
            # to how different 1/100.0001 and 1/100 are.
            if abs(A[i, j]) > abs(A[pivot, j]) :
                pivot = i

        A[[row_to_check, pivot], :] = A[[pivot, row_to_check], :]

        r = row_to_check  # for notational clarity
        Q = np.eye(n)
        Q[r, r] = 0.
        Q[pivot, pivot] = 0.
        Q[r, pivot] = 1.
        Q[pivot, r] = 1.
        Q_list.append(Q.copy())

        L = np.eye(n)

        if abs(A[r, j]) < eps :
            # All candidates for the pivots are essentially zero.
            # We cannot perform elimination with this column, so we skip.
            pass
        else :
            for k in range(r+1, n):
                u = A[k, j] / A[r, j]
                A[k, j:] -= u * A[j, j:]
                L[k, j] = -u
            row_to_check += 1  # next time we should start from the row below.

        L_list.append(L.copy())

        if verbose :
            print("After step", j+1, ":")
            print(A)
            print()

    return A, Q_list, L_list

In [13]:
U, Q_list, L_list = elimination(A, eps=1e-12, verbose=True)

After step 1 :
[[ 9.      9.      0.      4.      7.      3.      2.      7.      2.      0.    ]
 [ 0.     -1.      8.      4.8889 -4.4444  3.6667  5.4444  1.5556  6.4444  1.    ]
 [ 0.      4.      8.      6.7778  0.1111  1.3333 -1.1111 -0.8889  3.8889  0.    ]
 [ 0.      1.      8.      0.1111  1.4444  2.3333  2.5556  5.4444 -0.4444  1.    ]
 [ 0.     -5.      3.      0.7778  3.1111  7.3333  1.8889  1.1111  0.8889  4.    ]
 [ 0.      4.      5.      5.      6.      8.      4.      1.      4.      9.    ]
 [ 0.     -7.      1.      3.4444  2.7778  6.3333  1.2222 -0.2222  5.2222  2.    ]
 [ 0.      3.      5.      9.      4.      4.      6.      4.      4.      3.    ]
 [ 0.      0.      8.      2.2222 -0.1111  5.6667  4.1111  1.8889 -0.8889  1.    ]
 [ 0.      4.      3.     -2.2222  1.1111 -1.6667 -0.1111 -1.8889  2.8889  2.    ]]

After step 2 :
[[ 9.      9.      0.      4.      7.      3.      2.      7.      2.      0.    ]
 [ 0.     -7.      1.      3.4444  2.7778  6.3333  1.22

Let us reconstruct $Q$ and $L$ for $Q_j$'s and $L_j$'s. 

In [14]:
Lprime = L_list[n-1]
Q = np.eye(n)

for i in range(n-1, 0, -1) :
    Q = Q @ Q_list[i]
    Lprime_i = Q @ L_list[i-1] @ Q.T
    Lprime = Lprime @ Lprime_i
Q = Q @ Q_list[0]

print('A = ')
print(A)
print('Q.T @ L\'^{-1} @ U = ')
print(Q.T @ np.linalg.inv(Lprime) @ U)
print('Q = ')
print(Q)
print('L\' = ')
print(Lprime)

A = 
[[5. 0. 3. 3. 7. 9. 3. 5. 2. 4.]
 [7. 6. 8. 8. 1. 6. 7. 7. 8. 1.]
 [5. 9. 8. 9. 4. 3. 0. 3. 5. 0.]
 [2. 3. 8. 1. 3. 3. 3. 7. 0. 1.]
 [9. 9. 0. 4. 7. 3. 2. 7. 2. 0.]
 [0. 4. 5. 5. 6. 8. 4. 1. 4. 9.]
 [8. 1. 1. 7. 9. 9. 3. 6. 7. 2.]
 [0. 3. 5. 9. 4. 4. 6. 4. 4. 3.]
 [4. 4. 8. 4. 3. 7. 5. 5. 0. 1.]
 [5. 9. 3. 0. 5. 0. 1. 2. 4. 2.]]
Q.T @ L'^{-1} @ U = 
[[ 5. -0.  3.  3.  7.  9.  3.  5.  2.  4.]
 [ 7.  6.  8.  8.  1.  6.  7.  7.  8.  1.]
 [ 5.  9.  8.  9.  4.  3.  0.  3.  5.  0.]
 [ 2.  3.  8.  1.  3.  3.  3.  7. -0.  1.]
 [ 9.  9.  0.  4.  7.  3.  2.  7.  2.  0.]
 [ 0.  4.  5.  5.  6.  8.  4.  1.  4.  9.]
 [ 8.  1.  1.  7.  9.  9.  3.  6.  7.  2.]
 [ 0.  3.  5.  9.  4.  4.  6.  4.  4.  3.]
 [ 4.  4.  8.  4.  3.  7.  5.  5.  0.  1.]
 [ 5.  9.  3.  0.  5.  0.  1.  2.  4.  2.]]
Q = 
[[0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0

We further compare this decomposition output with a result from numpy.linalg.lu function. We confirm that for this small example, they coincide.

In [15]:
Q1, L1, U1 = sp.linalg.lu(A)
print('Q from numpy.linalg.lu = ')
print(Q1.T)
print('L\' from numpy.linalg.lu = ')
print(np.linalg.inv(L1))

Q from numpy.linalg.lu = 
[[0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
L' from numpy.linalg.lu = 
[[ 1.      0.      0.      0.      0.      0.      0.      0.      0.      0.    ]
 [-0.8889  1.      0.      0.      0.      0.      0.      0.      0.      0.    ]
 [-1.0635  0.5714  1.      0.      0.      0.      0.      0.      0.      0.    ]
 [ 0.6611 -0.4    -0.95    1.      0.      0.      0.      0.      0.      0.    ]
 [ 0.2934  0.1334 -0.8082  0.1665  1.      0.      0.      0.      0.      0.    ]
 [ 0.3062 -0.3457 -1.2765 -0.3037  0.9975  1.      0.      0.      0.      0.    ]
 [ 0.6414 -0.4087 -1.2013  0.4159 -0.2725  0.3818  1.      0.      0.      0.    ]
 [-0.9612  0.411  -0.0209 -0.6254 -



---

